<a id='Top'></a>
# Analytics Programming: Module 10
## Aggregating NYC Vehicle Crash Data
#### Alan Leidner Nov 27, 2019

In this notebook I will create visualizations of data from the NYC Crashes Data set.

I will import the data, and then attempt to create visualizations from therectangular data. Through these images, I hope to give readers an understanding of the relative and signifigant danger to life presented by current drivers in NYC. I will attempt to compare fatalaties across boroughs to give the reader a sense of the relative danger, but then intend to map each fatality over time on an interactive map, to present a granular view of the data.

1. [Importing](#Importing)
2. [Borough Visualization](#Viz1)
    - [Slice Dataframe](#slice_data)
    - [Graph Algebra](#Graph1)
3. [Map Visualization](#Viz2)
    - [Slice Dataframe](#slice_data2)
    - [Interactive Heat Map](#Map)
4. [Closing Thoughts](#conclusion)
    

DataSource: https://data.cityofnewyork.us/resource/h9gi-nx95

### Importing the data<a id='Importing'></a>
First I will import the data using the Socrata API from the website.
<br> I will import it into a dataframe which will let us read the csv data.

In [1]:
import pandas as pd
import datetime as dt
pd.set_option('display.max_colwidth', -1) # shows full cell text, instead of  truncated data in cells
pd.set_option('display.max.columns', None) # Let's me see all columns when calling a dataframe
crashes = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=20000")

In [2]:
pd.concat([crashes.head(), crashes.tail()], axis=0)

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-08-23T00:00:00.000,8:00,BRONX,10456.0,40.832500,-73.904400,POINT (-73.9044 40.8325),NaN,NaN,540 EAST 169 STREET,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4195346,Sedan,NaN,NaN,NaN,NaN
1,2019-09-13T00:00:00.000,16:00,NaN,NaN,40.722496,-73.940414,POINT (-73.940414 40.722496),BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0,0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,4205878,Flat Bed,Pick-up Truck,NaN,NaN,NaN
2,2019-09-02T00:00:00.000,13:00,BROOKLYN,11233.0,40.680900,-73.928650,POINT (-73.92865 40.6809),NaN,NaN,321 REID AVENUE,0,0,0,0,0,0,0,0,Driver Inexperience,Unspecified,NaN,NaN,NaN,4199096,Sedan,NaN,NaN,NaN,NaN
3,2019-09-16T00:00:00.000,5:10,STATEN ISLAND,10308.0,40.553150,-74.155290,POINT (-74.15529 40.55315),NaN,NaN,162 COLON AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4206461,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-08-23T00:00:00.000,15:30,MANHATTAN,10001.0,40.749054,-73.995760,POINT (-73.99576 40.749054),8 AVENUE,WEST 29 STREET,NaN,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,Unspecified,NaN,NaN,4193774,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Taxi,NaN,NaN
19995,2018-04-25T00:00:00.000,5:25,BRONX,10465.0,40.830100,-73.837760,POINT (-73.83776 40.8301),NaN,NaN,2505 BRUCKNER BOULEVARD,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,3892060,Sedan,Sedan,NaN,NaN,NaN
19996,2018-05-05T00:00:00.000,12:00,BROOKLYN,11211.0,40.711480,-73.945600,POINT (-73.9456 40.71148),GRAND STREET,MANHATTAN AVENUE,NaN,1,0,0,0,1,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,3896011,SPORT UTILITY / STATION WAGON,BICYCLE,NaN,NaN,NaN
19997,2018-05-11T00:00:00.000,7:45,BROOKLYN,11233.0,40.669476,-73.919975,POINT (-73.919975 40.669476),EASTERN PARKWAY,HOWARD AVENUE,NaN,0,0,0,0,0,0,0,0,Other Vehicular,Unspecified,NaN,NaN,NaN,3898623,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
19998,2018-04-25T00:00:00.000,1:13,NaN,NaN,40.709442,-73.960700,POINT (-73.9607 40.709442),BROADWAY,NaN,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,3887698,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
19999,2018-05-10T00:00:00.000,18:30,BROOKLYN,11212.0,40.670006,-73.910840,POINT (-73.91084 40.670006),NaN,NaN,414 ROCKAWAY AVENUE,0,0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,3898280,Sedan,Pick-up Truck,NaN,NaN,NaN


In [3]:
crashes.shape

(20000, 29)

In [4]:
crashes['accident_date'] = pd.to_datetime(crashes['accident_date']) #makes the date easier to work with later
crashes

,accident_date,accident_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2019-08-23,8:00,BRONX,10456.0,40.832500,-73.904400,POINT (-73.9044 40.8325),NaN,NaN,540 EAST 169 STREET,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4195346,Sedan,NaN,NaN,NaN,NaN
1,2019-09-13,16:00,NaN,NaN,40.722496,-73.940414,POINT (-73.940414 40.722496),BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0,0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,4205878,Flat Bed,Pick-up Truck,NaN,NaN,NaN
2,2019-09-02,13:00,BROOKLYN,11233.0,40.680900,-73.928650,POINT (-73.92865 40.6809),NaN,NaN,321 REID AVENUE,0,0,0,0,0,0,0,0,Driver Inexperience,Unspecified,NaN,NaN,NaN,4199096,Sedan,NaN,NaN,NaN,NaN
3,2019-09-16,5:10,STATEN ISLAND,10308.0,40.553150,-74.155290,POINT (-74.15529 40.55315),NaN,NaN,162 COLON AVENUE,0,0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4206461,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
4,2019-08-23,15:30,MANHATTAN,10001.0,40.749054,-73.995760,POINT (-73.99576 40.749054),8 AVENUE,WEST 29 STREET,NaN,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,Unspecified,NaN,NaN,4193774,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,Taxi,NaN,NaN
5,2019-09-15,11:30,QUEENS,11413.0,40.676758,-73.743220,POINT (-73.74322 40.676758),NaN,NaN,134-15 227 STREET,0,0,0,0,0,0,0,0,Backing Unsafely,Unspecified,NaN,NaN,NaN,4205885,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
6,2019-09-17,7:19,BRONX,10467.0,40.884407,-73.857640,POINT (-73.85764 40.884407),NaN,NaN,834 EAST 222 STREET,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4210598,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
7,2019-08-29,16:40,BRONX,10458.0,40.852528,-73.884346,POINT (-73.884346 40.852528),NaN,NaN,2323 CROTONA AVENUE,1,0,0,0,0,0,1,0,Unsafe Speed,Passing or Lane Usage Improper,NaN,NaN,NaN,4196957,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
8,2019-08-20,6:45,BROOKLYN,11212.0,40.658195,-73.925980,POINT (-73.92598 40.658195),NaN,NaN,269 REMSEN AVENUE,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,4191949,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
9,2019-09-08,1:10,NaN,NaN,40.804700,-73.912430,POINT (-73.91243 40.8047),BRUCKNER BOULEVARD,EAST 138 STREET,NaN,1,0,0,0,0,0,1,0,Alcohol Involvement,NaN,NaN,NaN,NaN,4202783,Sedan,NaN,NaN,NaN,NaN


# Using plotly and/or bokeh, make two data visualizations, at least one of which is interactive or animated. "Interactive" means that the viewer can change the data visualization by doing something like changing a widget value, zooming in, or clicking on a data point. "Animated" means that with or without user interaction, the data visualization moves and changes to represent some change across data (like change in time).

### Borough Visualization <a id='Viz1'></a><br><div style="text-align: right">[Begining of the page](#Top)</div>

Group crashes by borough and month, along with the sum of persons killed. <a id='slice_data'>

In [34]:
crashes_by_month = crashes.groupby(['borough', pd.Grouper(key='accident_date', freq='M')]).number_of_persons_killed.sum().copy()
crashes_by_month

borough        accident_date
BRONX          2018-02-28       0
               2018-04-30       0
               2018-05-31       0
               2019-05-31       0
               2019-06-30       0
               2019-08-31       1
               2019-09-30       1
               2019-10-31       0
               2019-11-30       0
BROOKLYN       2017-10-31       0
               2017-11-30       0
               2018-02-28       0
               2018-03-31       0
               2018-04-30       0
               2018-05-31       0
               2018-09-30       0
               2019-05-31       0
               2019-06-30       0
               2019-08-31       1
               2019-09-30       4
               2019-10-31       0
               2019-11-30       0
MANHATTAN      2017-10-31       0
               2018-02-28       0
               2018-03-31       0
               2018-04-30       0
               2018-05-31       0
               2019-08-31       0
               2019

In [35]:
crashes_by_month

borough        accident_date
BRONX          2018-02-28       0
               2018-04-30       0
               2018-05-31       0
               2019-05-31       0
               2019-06-30       0
               2019-08-31       1
               2019-09-30       1
               2019-10-31       0
               2019-11-30       0
BROOKLYN       2017-10-31       0
               2017-11-30       0
               2018-02-28       0
               2018-03-31       0
               2018-04-30       0
               2018-05-31       0
               2018-09-30       0
               2019-05-31       0
               2019-06-30       0
               2019-08-31       1
               2019-09-30       4
               2019-10-31       0
               2019-11-30       0
MANHATTAN      2017-10-31       0
               2018-02-28       0
               2018-03-31       0
               2018-04-30       0
               2018-05-31       0
               2019-08-31       0
               2019

In [36]:
crashes_by_month.dtypes()

TypeError: 'numpy.dtype' object is not callable

and now to graph <a id='Graph1'>

In [37]:
import plotly.express as px
#fig = px.line(crashes_by_month,x ='accident_date',y ='number_of_persons_killed', color='borough')
#fig.show()
fig = px.box(crashes_by_month,  x="borough", y="accident_date")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['number_of_persons_killed'] but received: borough

### Map Visualization <a id='Viz2'></a><br><div style="text-align: right">[Begining of the page](#Top)</div>

We will have to prepare a new dataframe with the lat/long values, before this can work. <a id='slice_data2'>

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(go.Densitymapbox(lat=quakes.Latitude, lon=quakes.Longitude, z=quakes.Magnitude, radius=10))
fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=180)
fig.update_layout(margin={"r":1,"t":0,"l":0,"b":0})
fig.show()<a id='Map'>

### Conclusion <a id='conclusion'></a><br><div style="text-align: right">[Begining of the page](#Top)</div>